# Kong: Performance Evaluation

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os

os.environ["PNML2NUPN"] = "~/pnml2nupn-3.2.0.jar"

In [4]:
MODELS_URL = "https://mcc.lip6.fr/2019/archives/2019-mcc-models.tar.gz"
os.system("wget {}".format(MODELS_URL))

#  wget -r --no-parent -A '*-pnml.tar.gz' https://mcc.lip6.fr/archives/
# ls *.tar.gz |xargs -n1 tar -xvf
# rm *.tar.gz


0